In [6]:
import numpy as np

class BackPropagationNetwork:
    def __init__(self, layerSize):
        self.layerCount = len(layerSize) - 1  # Number of layers (excluding input layer)
        self.shape = layerSize  # The structure of the neural network
        self.weights = []  # List to store weights for each layer
        self._layerInput = []  # Input to each layer
        self._layerOutput = []  # Output from each layer

        # Initialize weights with random values using a normal distribution
        for (l1, l2) in zip(layerSize[:-1], layerSize[1:]):
            self.weights.append(np.random.normal(scale=0.1, size=(l2, l1 + 1)))

    def sgm(self, x, Derivative=False):
        """Sigmoid activation function and its derivative"""
        if not Derivative:
            return 1 / (1 + np.exp(-x))  # Sigmoid function
        else:
            out = self.sgm(x)
            return out * (1 - out)  # Derivative of sigmoid

    def run(self, input):
        """Run the network with given input and compute outputs"""
        inCases = input.shape[0]  # Number of input cases
        self._layerInput = []
        self._layerOutput = []

        # Forward propagation through all layers
        for index in range(self.layerCount):
            if index == 0:
                # For the first layer, include input and bias
                layerInput = self.weights[index].dot(np.vstack([input.T, np.ones([1, inCases])]))
            else:
                # For subsequent layers, use previous layer's output as input
                layerInput = self.weights[index].dot(np.vstack([self._layerOutput[-1], np.ones([1, inCases])]))
            self._layerInput.append(layerInput)
            self._layerOutput.append(self.sgm(layerInput))

        # Return the final output (transpose to match input format)
        return self._layerOutput[-1].T

    def trainEpoch(self, Input, target, trainingRate=0.2):
        """Train the network for one epoch using backpropagation"""
        delta = []  # List to store errors (deltas) for each layer
        inCases = Input.shape[0]  # Number of input cases
        self.run(Input)  # Perform forward propagation
        error = 0  # Initialize total error

        # Backpropagation of errors
        for index in reversed(range(self.layerCount)):
            if index == self.layerCount - 1:  
                # For the output layer
                output_delta = self._layerOutput[index] - target.T
                error = np.sum(output_delta ** 2)  # Mean Squared Error
                delta.append(output_delta * self.sgm(self._layerInput[index], True))
            else:
                # For hidden layers, propagate errors backwards
                delta_pullback = self.weights[index + 1].T.dot(delta[-1])
                delta.append(delta_pullback[:-1, :] * self.sgm(self._layerInput[index], True))

        # Update weights using the computed deltas
        for index in range(self.layerCount):
            delta_index = self.layerCount - 1 - index  # Reverse the order of deltas
            if index == 0:
                # Input layer: include input and bias
                layerOutput = np.vstack([Input.T, np.ones([1, inCases])])
            else:
                # Hidden layers: use previous layer's output and bias
                layerOutput = np.vstack([self._layerOutput[index - 1], np.ones([1, self._layerOutput[index - 1].shape[1]])])

            # Compute weight delta and update weights
            weightDelta = np.sum(
                layerOutput[None, :, :].transpose(2, 0, 1) * delta[delta_index][None, :, :].transpose(2, 1, 0), axis=0)
            self.weights[index] -= trainingRate * weightDelta

        return error


if __name__ == "__main__":
    # Create a BackPropagationNetwork instance with a (2,2,1) structure
    bpn = BackPropagationNetwork((2, 2, 1))
    print("Network Structure:", bpn.shape)
    print("Initial Weights:", bpn.weights)

    # Input and target for XOR problem
    lvInput = np.array([[0, 0], [1, 1], [0, 1], [1, 0]])
    lvTarget = np.array([[0.05], [0.05], [0.95], [0.95]])

    lnMax = 100000  # Maximum number of iterations
    lnError = 1e-5  # Error threshold

    # Training loop
    for i in range(lnMax + 1):
        err = bpn.trainEpoch(lvInput, lvTarget)
        if i % 5000 == 0:
            print("Iteration {0}\tError {1:0.6f}".format(i, err))
        if err <= lnError:
            print("Minimum Error reached at iteration: {0}".format(i))
            break

    # Run the trained network on the input data
    lvOutput = bpn.run(lvInput)
    print("Input: {0}\n\nOutput: {1}".format(lvInput, lvOutput))


Network Structure: (2, 2, 1)
Initial Weights: [array([[ 0.17365166,  0.14824008,  0.04162851],
       [ 0.02975941,  0.05697061, -0.01814336]]), array([[-0.00912843, -0.10510329,  0.03960442]])]
Iteration 0	Error 0.810089
Iteration 5000	Error 0.809988
Iteration 10000	Error 0.809689
Iteration 15000	Error 0.007611
Iteration 20000	Error 0.000349
Iteration 25000	Error 0.000045
Minimum Error reached at iteration: 29169
Input: [[0 0]
 [1 1]
 [0 1]
 [1 0]]

Output: [[0.05186594]
 [0.05156521]
 [0.94857523]
 [0.9485747 ]]
